In [1]:
import numpy as np
from scipy.signal import convolve2d

In [2]:
# importing libraries
from mpl_toolkits import mplot3d
import numpy as np
import matplotlib.pyplot as plt
%matplotlib widget


In [3]:
PI = np.pi
dx = 1/25
dy = 1/25
x_axis,y_axis = np.arange(0,1,dx), np.arange(0,1,dy) # axes
shape = (len(x_axis),len(y_axis))
X = np.outer(x_axis,np.ones(shape[1]))
Y = np.outer(np.ones(shape[0]),y_axis)

In [4]:
# Fix boundary conditions
def fix_boundary_0(sol):
    sol[0,:] = np.zeros(shape[1])
    sol[-1,:] = Y[0,:]
    sol[:,0] = sol[:,1]
    sol[:,-1] = sol[:,-2]
    return # sol is a pointer no need to return anything

def fix_boundary_1(sol):
    sol[0,:] = np.cos(4*PI*Y[0,:]) # Dirichlet
    sol[-1,:] = -np.cos(4*PI*Y[0,:]) # Dirichlet
    sol[:,0] =  sol[:,1] - dx*3.0 # Neuman, slopy
    sol[:,-1] = sol[:,-2] - dx*3.0 # Neuman
    
def fix_boundary_2(sol):
    sol[0,:] = np.cos(8*PI*Y[0,:]) # Dirichlet
    sol[-1,:] = np.sin(8*PI*Y[0,:]) # Dirichlet
    sol[:,0] =  sol[:,1] - dx*3.0*np.cos(4*PI*X[:,0]) # Neuman, slopy
    sol[:,-1] = sol[:,-2] - dx*3.0*np.sin(4*PI*X[:,0]) # Neuman

In [5]:
# Initiate with Zeros
sol = np.zeros(shape)

# Define which boundary conditions you want
fix_boundary = fix_boundary_2

fix_boundary(sol)

### Solve the system

# Make 10 itermaions toward solving the solution
def solve_iter(sol,ker,poisson_source=None):
    if poisson_source is None: poisson_source = np.zeros(sol.shape) # solve laplacian
    sol[1:-1,1:-1] = convolve2d(sol,ker,mode="valid")
    sol += poisson_source
    fix_boundary(sol)
def get_err(sol,ker,poisson_source=None): # estimate the error
    sol_next = sol.copy() # deep copy
    solve_iter(sol_next,ker,poisson_source)
    return np.mean((sol-sol_next).flatten()**2) 
    
# Initiate the kernel
kx = dx**2 / (dx**2 + dy**2)
ky = dy**2 / (dx**2 + dy**2)
ker = np.array([[0,kx,0],[ky,0,ky],[0,kx,0]]) * 0.5
poisson_source = np.zeros(shape)
poisson_source[shape[0]//4,shape[1]//4] = 5
poisson_source[3*shape[0]//4,3*shape[1]//4] = -5

# Plot each iteration
def plot_sol(X,Y,z,err):
    fig = plt.figure()
 
    # syntax for 3-D plotting
    ax = plt.axes(projection ='3d')

    # syntax for plotting
    ax.plot_surface(X, Y, z, cmap ='viridis') # edgecolo="green"
    ax.set_title(f'Solution, iteration: Err = {round(err,1)}')
    plt.show()
    
#     input("[Enter]")
plot_sol(X,Y,sol,np.log10(get_err(sol,ker)))

steps = 6000
for i in range(steps):
    solve_iter(sol,ker,poisson_source)
    if i % (steps//3) == 0:
        err = get_err(sol,ker,poisson_source)
        plot_sol(X,Y,sol,np.log10(err))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …